# Dependencies and Global Setup

In [34]:
# importing dependencies
from bs4 import BeautifulSoup
from splinter import Browser
import requests
import pymongo
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

In [35]:
# create driver and browser
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Driver [C:\Users\Mary\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


# Latest News Titles and Paragraph Scrape

In [36]:
# URL of news page to be scraped and visit it with browser
news_url = 'https://mars.nasa.gov/news/'
browser.visit(news_url)

In [37]:
# HTML object
html = browser.html

# initiating soup object for news scrape
news_soup = BeautifulSoup(html, 'html.parser')
slide_elem = news_soup.select_one('ul.item_list li.slide')

In [38]:
# getting title and paragraph for latest article
title = slide_elem.find('div', class_='content_title').text
paragraph = slide_elem.find('div', class_='article_teaser_body').get_text()

print(title)
print(paragraph)

NASA's Perseverance Pays Off Back Home
Even as the Perseverance rover approaches Mars, technology on board is paying off on Earth.


# Featured Image URL Scrape

In [44]:
# URL of image page to be scraped and visit it with browser
space_image_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(space_image_url)

In [47]:
# HTML object
html = browser.html
# instatiating beautiful soup object and parsing using lxml
image_soup = BeautifulSoup(html, 'lxml')

# accessing the image url
image = image_soup.find('a', class_='showimg fancybox-thumbs')['href']

# slicing the page url
new_space_image_url = space_image_url[0:-10]

# attaching the image url to sliced page url
image_url = f"{new_space_image_url}{image}"

print(image_url)

https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars1.jpg


# Mars Facts Table Scrape

In [90]:
# URL of facts page to be scraped and visit it with browser
facts_url = 'https://space-facts.com/mars/'

# parsing html using pandas
facts_tables = pd.read_html(facts_url)
facts_tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [77]:
# turning pandas parse into a dataframe
facts_df = facts_tables[0]

# cleaning up the table for printing
facts_df = facts_df.rename(columns={0: " ", 1: "Mars"})
facts_df.set_index(" ", inplace=True)

# converting df to html with pandas
facts_html = facts_df.to_html()

# Mars Hemisphere Scraping

In [78]:
hemi_image_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemi_image_url)

In [97]:
# HTML object
html = browser.html
# instatiating beautiful soup object and parsing with html
hemi_image_soup = BeautifulSoup(html, 'html.parser')

# accessing the names of the hemispheres and storing in a list
hemi_name_list = []
hemi_names = hemi_image_soup.find_all('div', class_='description')
for name in hemi_names:
    hemi_name_list.append(name.a.h3.text)


print(hemi_name_list)

['Cerberus Hemisphere Enhanced', 'Schiaparelli Hemisphere Enhanced', 'Syrtis Major Hemisphere Enhanced', 'Valles Marineris Hemisphere Enhanced']
